### Libraries

In [36]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import Adam

### Settings

In [2]:
### Paths ###

# IP102 path
ip_102_path = "../../../Data/MSiA 432/03_hw/IP102-Dataset/"

# FairFace path
fair_face_path = "../../../Data/MSiA 432/03_hw/FairFace/"

# Question 1

In word document attatched with this submission

# Question 2

In [25]:
columns = ["img_path", "target"]

train_df = pd.read_csv(ip_102_path + "train.txt", header=None, sep=" ", names=columns)
val_df = pd.read_csv(ip_102_path + "val.txt", header=None, sep=" ", names=columns)
test_df = pd.read_csv(ip_102_path + "test.txt", header=None, sep=" ", names=columns)

for df in [train_df, val_df, test_df]:
    df.target = df.target.astype(str)

# Get # unique classes
num_classes = len(train_df.target.unique())

train_df.head()

,img_path,target
0,00002.jpg,0
1,00003.jpg,0
2,00005.jpg,0
3,00006.jpg,0
4,00008.jpg,0


In [26]:
# Define ImageDataGenerator for training (with augmentation)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest")

# Define ImageDataGenerator for validation and testing (without augmentation)
valid_test_datagen = ImageDataGenerator(rescale=1./255)

In [27]:
batch_size = 8
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=ip_102_path + "/classification/train/",
    x_col="img_path",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42, 
    validate_filenames=False
)
valid_generator = valid_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=ip_102_path + "/classification/val/",
    x_col="img_path",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    validate_filenames=False
)
test_generator = valid_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=ip_102_path + "/classification/test/",
    x_col="img_path",
    target_size=(100, 100),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    validate_filenames=False
)

Found 45095 non-validated image filenames belonging to 102 classes.
Found 7508 non-validated image filenames belonging to 102 classes.
Found 22619 non-validated image filenames.


In [32]:
optimizer = "adam"
loss = "categorical_crossentropy"
metrics = ["accuracy", "roc_auc"]

In [33]:
def create_model():

    # Define CNN model
    model = Sequential([

    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax'),

    ])

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

model = create_model()

In [41]:
num_gpu = len(tf.config.list_physical_devices('GPU'))
print(f"{num_gpu} GPU(s) available")

1 GPU(s) available


In [34]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // 32,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=len(val_df) // 32
)

FileNotFoundError: [Errno 2] No such file or directory: '../../../Data/MSiA 432/03_hw/IP102-Dataset//classification/train/57099.jpg'

# Question 3

# Question 4